In [14]:
from bs4 import BeautifulSoup
from urllib import request, error
from urllib.parse import urljoin, urlparse

import itertools
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [15]:
def parse_url(url):
    try:
        print(url)
        html = request.urlopen(url)
        soup = BeautifulSoup(html, "html.parser")
        frame_srcs = [f["src"] for f in soup.find_all("frame")]
        if len(frame_srcs) == 0:
            return [soup]
        url_parts = urlparse(url)
        domain = url_parts.scheme + "://" + url_parts.netloc
        soups = [soup]
        for src in frame_srcs:
            path = urlparse(src)
            site_url = urljoin(domain, path.path) if path.scheme == "" else path.geturl()
            soups.append(parse_url(site_url))
        return list(itertools.chain.from_iterable(soups))
    except error.HTTPError as err:
        print(err)
        return []

In [16]:
def parse_tags(soups):
    tags = []
    for s in soups:
        tag = [tag.name for tag in s.find_all()]
        tags.append(tag)
    return list(itertools.chain.from_iterable(tags))

In [17]:
def to_tag_df(tags):
    return pd.DataFrame.from_records([[tag] for tag in tags], columns=["tag"])

In [18]:
LEGACY_SITES = [
    "http://abehiroshi.la.coocan.jp/",
    "http://www.takenet.or.jp/~hiro/",
    "http://www2u.biglobe.ne.jp/~akichan/",
    "http://www.geocities.co.jp/Milkyway-Sirius/9160/",
    "http://shojicam.com/",
    "http://www.ogaemon.com/",
    "http://museum.scenecritique.com/home/idoldream/index.html",
    "http://manpukuotahuku.seesaa.net/article/455698656.html",
]

In [20]:
leg_soups = [parse_url(url) for url in LEGACY_SITES]
leg_tags = [parse_tags(s) for s in leg_soups]
leg_dfs = [to_tag_df(tags) for tags in leg_tags]

http://abehiroshi.la.coocan.jp/
http://abehiroshi.la.coocan.jp/menu.htm
http://abehiroshi.la.coocan.jp/top.htm
http://www.takenet.or.jp/~hiro/
http://www.takenet.or.jp/~hiro/index2.html


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://www.takenet.or.jp/~hiro/menew.html
http://www2u.biglobe.ne.jp/~akichan/
http://www.geocities.co.jp/Milkyway-Sirius/9160/
http://shojicam.com/
http://www.geocities.jp/leica_collecting/
http://www.geocities.jp/header04.htm
HTTP Error 404: Not Found
http://www.geocities.jp/navi04.html
HTTP Error 404: Not Found
http://www.geocities.jp/top04.html
HTTP Error 404: Not Found
http://shojicam.com/blank.html
http://www.ogaemon.com/
http://museum.scenecritique.com/home/idoldream/index.html
http://manpukuotahuku.seesaa.net/article/455698656.html


AttributeError: 'NavigableString' object has no attribute 'find_all'

In [ ]:
soup.find_all("frame")